In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [14]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

def read_image(filename):
    img = Image.open(filename).convert('L')  # Convert to grayscale
    return np.array(img)

def vectorize_patches(image, patch_size=(8, 8)):
    patches = []
    height, width = image.shape
    for y in range(0, height, patch_size[0]):
        for x in range(0, width, patch_size[1]):
            patch = image[y:y+patch_size[0], x:x+patch_size[1]].flatten()
            patches.append(patch)
#             print(patch.shape)
    print(patches)
    print(len(patches), len(patches[0]))
    return patches
#     return np.array(patches)


def zero_mean(data):
    mean = np.mean(data, axis=0)
    return data - mean, mean

def covariance_matrix(data):
    n = data.shape[0]
    return (1 / n) * np.dot(data.T, data)

def pca(data, num_components):
    cov_matrix = covariance_matrix(data)
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)
    sorted_indices = np.argsort(eigenvalues)[::-1]
    top_indices = sorted_indices[:num_components]
    return eigenvectors[:, top_indices]

def reconstruct_image(patches, components, mean, patch_size=(8, 8)):
    reconstructed_patches = np.dot(patches, components)
    reconstructed_patches += mean
    reconstructed_image = np.zeros_like(patches)
    height, width = reconstructed_image.shape
    for i in range(0, height, patch_size[0]):
        for j in range(0, width, patch_size[1]):
            patch = reconstructed_patches[i * width + j].reshape(patch_size)
            reconstructed_image[i:i+patch_size[0], j:j+patch_size[1]] = patch
    return reconstructed_image

def main():
    # Read the image
    image = read_image("lighthouse.png")
    print(image)
    # Vectorize the image patches
    patches = vectorize_patches(image)
    
    # Normalize the data
    patches, mean = zero_mean(patches)
    
    # Perform PCA for different compression levels
    for compression_level in [48, 32, 16]:
        # Perform PCA
        components = pca(patches, compression_level)
        
        # Reconstruct the image
        reconstructed_image = reconstruct_image(patches, components, mean)
        
        # Clip pixel values to valid range [0, 255]
        reconstructed_image = np.clip(reconstructed_image, 0, 255).astype(np.uint8)
        
        # Display the reconstructed image
        plt.imshow(reconstructed_image, cmap='gray')
        plt.title(f"Compression Level: {compression_level}")
        plt.axis('off')
        plt.show()

if __name__ == "__main__":
    main()


[[114 113 112 ... 136 138 209]
 [112 111 112 ... 140 143 211]
 [113 112 111 ... 144 144 212]
 ...
 [209 208 207 ... 208 207 236]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
[array([114, 113, 112, 114, 115, 116, 114, 112, 112, 111, 112, 111, 112,
       112, 112, 111, 113, 112, 111, 111, 112, 112, 111, 110, 113, 113,
       112, 112, 112, 112, 111, 110, 116, 114, 112, 113, 113, 114, 112,
       111, 117, 116, 114, 114, 114, 115, 112, 111, 118, 116, 115, 115,
       116, 116, 114, 112, 115, 114, 112, 112, 112, 113, 112, 111],
      dtype=uint8), array([111, 114, 115, 113, 113, 111, 112, 111, 110, 111, 112, 112, 112,
       111, 112, 111, 108, 111, 111, 110, 110, 110, 110, 110, 108, 109,
       109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109,
       109, 109, 108, 108, 108, 108, 108, 108, 108, 109, 108, 107, 107,
       107, 107, 107, 108, 109, 105, 103, 103, 103, 103, 103, 104],
      dtype=uint8), array([111, 110, 110, 110, 113, 114, 113, 112, 110, 109

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (7569,) + inhomogeneous part.